##4.2 For Statements

In [1]:
words = ['cat', 'window', 'defenestrate']
for w in words:
    print(w, len(w))

cat 3
window 6
defenestrate 12


在疊代一個集合的同時修改該集合的內容，很難獲取想要的結果。比較直觀的替代方式，是疊代該集合的副本，或建立一個新的集合：

In [2]:
# Create a sample collection
users = {'Sherry': 'active', 'Jason': 'inactive', 'Ruth': 'active'}
print(users.keys())
print(users.items())

dict_keys(['Sherry', 'Jason', 'Ruth'])
dict_items([('Sherry', 'active'), ('Jason', 'inactive'), ('Ruth', 'active')])


In [3]:
for name, status in users.items():
  if status == 'inactive':
    del users[name]

print(users)

RuntimeError: dictionary changed size during iteration

In [4]:
# Strategy(1): iterate over a copy
for name, status in users.copy().items():
  if status == 'inactive':
    del users[name]

print(users)

{'Sherry': 'active', 'Ruth': 'active'}


In [5]:
users = {'Sherry': 'active', 'Jason': 'inactive', 'Ruth': 'active'}

# Strategy(2): Create a new collection
active_users = {}
for name, status in users.items():
  if status == 'active':
    active_users[name] = status

print(active_users)

{'Sherry': 'active', 'Ruth': 'active'}


##4.3 range() function
在很多情況下，由 range() 回傳的物件表現得像是一個 list（串列）一樣，但實際上它並不是。它是一個在疊代時能夠回傳所要求的序列中所有項目的物件，但它不會真正建出這個序列的 list，以節省空間。
我們稱這樣的物件為 iterable（可疊代物件），意即能作為函式及架構中可以一直獲取項目直到取盡的對象。

In [6]:
for i in range(5):
  print(i)

0
1
2
3
4


In [7]:
list(range(5, 10))

[5, 6, 7, 8, 9]

In [8]:
list(range(0, 10, 3))

[0, 3, 6, 9]

In [9]:
list(range(-10, -100, -20))

[-10, -30, -50, -70, -90]

##4.6 match statements

In [10]:
def http_error(status):
    match status:
        case 400:
            return "Bad request"
        case 404:
            return "Not found"
        case 418 | 518:
            return "I'm a teapot"
        case _:
            return "Something's wrong with the internet"

In [11]:
http_error(520)

"Something's wrong with the internet"

In [12]:
#point is an (x, y) tuple
def test_point(point):
  match point:
    case (0, 0):
        print("Origin")
    case (0, y):
        print(f"Y={y}")
    case (x, 0):
        print(f"X={x}")
    case (x, y):
        print(f"X={x}, Y={y}")
    case _:
        raise ValueError("Not a point")

In [13]:
test_point((0, 0))
test_point((5, 0))
test_point((1, 3))
test_point((2))

Origin
X=5
X=1, Y=3


ValueError: Not a point

In [14]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

def where_is(point):
    match point:
        case Point(x=0, y=0):
            print("Origin")
        case Point(x=0, y=y):
            print(f"Y={y}")
        case Point(x=x, y=0):
            print(f"X={x}")
        case Point():
            print("Somewhere else")
        case _:
            print("Not a point")

In [15]:
where_is(Point(0, 0))
where_is(Point(1, 0))
where_is(Point(2, 3))
where_is((1, 2))

Origin
X=1
Somewhere else
Not a point


In [18]:
from enum import Enum
class Color(Enum):
    RED = 'red'
    GREEN = 'green'
    BLUE = 'blue'

color = Color(input("Enter your choice of 'red', 'blue' or 'green': "))

match color:
    case Color.RED:
        print("I see red!")
    case Color.GREEN:
        print("Grass is green")
    case Color.BLUE:
        print("I'm feeling the blues :(")

Enter your choice of 'red', 'blue' or 'green': blue
I'm feeling the blues :(


In [19]:
test = Color('blue')
print(test)


Color.BLUE


##4.7 Defining Functions

* 函式執行時會建立一個新的符號表 (symbol table) 來儲存該函式內的區域變數 (local variable)。更精確地說，所有在函式內的變數賦值都會把該值儲存在一個區域符號表。然而，在引用一個變數時，會先從區域符號表開始搜尋，其次為外層函式的區域符號表，其次為全域符號表 (global symbol table)，最後為所有內建的名稱。因此，在函式中，全域變數及外層函式變數雖然可以被引用，但無法被直接賦值（除非全域變數是在 global 陳述式中被定義，或外層函式變數在 nonlocal 陳述式中被定義）。

The *execution* of a function introduces a *new symbol table used for the local variables of the function*. More precisely, all variable assignments in a function store the value in the *local symbol table*; whereas variable references first look in the local symbol table, then in the local symbol tables of enclosing functions, then in the global symbol table, and finally in the table of built-in names. Thus, global variables and variables of enclosing functions cannot be directly assigned a value within a function (unless, for global variables, named in a global statement, or, for variables of enclosing functions, named in a nonlocal statement), although they may be referenced.

* -> 變數尋找順序（local → enclosing → global → builtins）



In [20]:
# local 變數與尋找順序
x = 100 # Global symbol table

def foo():
    y = 200 # foo's local symbol table
    print(f"foo -> x: {x}") # Cannot find local -> find global x=100
    print(f"foo -> y: {y}") #

foo()

foo -> x: 100
foo -> y: 200


In [23]:
# Unbound local error
x = 100

def bar():
    print(x)    # try to access local x, but in this function has assignment for x, Python think x is local.

    x = 5       # declare local x -> UnboundLocalError for print(x)

bar()

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

In [27]:
# Use global to modify global variable
count = 0

def inc():
    global count    # Specify to use the variable "count" in global symbol tabl
    count += 1

inc()
inc()
print(f"count = {count}")


count = 2


In [30]:
# use nonlocal to modify the variables in enclosing functions
def outer():
    msg = "hello"

    def inner():
        nonlocal msg    # Specify to use the variable "msg" in outer()'s symbol table
        msg = "world"

    inner()
    print(f"after inner: {msg}")

outer()


after inner: world


In [31]:
# comparison
def outer():
    msg = "hello"

    def inner():
        msg = "world"   # this "msg" is in inner()'s symbol table

    inner()
    print(f"after inner: {msg}")   # this "msg" is in outer()'s symbol table

outer()

after inner: hello


* 在一個函式被呼叫的時候，實際傳入的參數（引數）會被加入至該函式的區域符號表。因此，引數傳入的方式為傳值呼叫 (call by value)（這裡傳遞的值永遠是一個物件的參照 (reference)，而不是該物件的值）。當一個函式呼叫別的函式或遞迴呼叫它自己時，在被呼叫的函式中會建立一個新的區域符號表。

The actual parameters (arguments) to a function call are introduced in the local symbol table of the called function when it is called; thus, arguments are passed using call by value (where the value is always an object reference, not the value of the object). When a function calls another function, or calls itself recursively, a new local symbol table is created for that call.

In [33]:
# Call by value(value: object reference) -> Pass by "object reference"
def modify(val, lst):
    val = 999       # rebind local variable "val"
    lst.append(4)   # By reference, operating on the same list. Modify the original object.

a = 1
b = [1, 2, 3]
modify(a, b)
print(a, b)

1 [1, 2, 3, 4]


Python 的引數傳遞模式：既不是傳值也不是純參考，而是「傳參考的值」:

Python 採用「以物件參考為值的呼叫方式」（pass-by-object-reference，或稱 call-by-sharing）。
換句話說，呼叫時傳給函式的是物件的參考（reference），但這個參考本身是以值的方式傳入的。

* 傳值呼叫 (call by value) vs. 傳參考呼叫 (call by reference)
1. 傳值呼叫: 傳遞的是「值」的複本，不論該值多大、是物件還是原始型別，函式內賦新值只會影響副本，不會改到呼叫端。
2. 傳參考呼叫: 傳遞的是「變數所在位置（記憶體位址）」的參考，函式透過這個參考操作外部變數本身，賦新值或修改內容都會直接反映到呼叫端。
-> Python 不符合嚴格的「純傳值」或「純傳參考」定義，而是：
傳參考的值：函式接到一個指向原物件的參考，這個參考本身又是用值的方式傳入。

最貼切的描述是：
Python 的引數傳遞是將參考當作值傳遞，也就是在呼叫時複製物件參考，但不複製物件本身。
1. 與 Symbol Table 的關係:
Python 將「變數名稱」與「物件參考」的對應，儲存在函式執行時所建立的 symbol table（實作上是高速的陣列與映射結構）中。
引數傳遞機制，其實就是在呼叫時把「參考」放到當次呼叫的 local symbol table 裡，讓後續的賦值與讀取都透過這張 table 完成。








* 實際上，即使一個函式缺少一個 return 陳述式，它亦有一個固定的回傳值。這個值稱為 None（它是一個內建名稱）。

In [35]:
def compute_sum(a, b):
    total = a + b
    print(total)

x = compute_sum(3, 4)

print(f"External received: {x}")

7
External received: None


None 常見應用:
- 用 None 當作「預設值」或「尚未初始化」的標記
- 別把 None 誤認為空字串 ''、數字 0 或空列表 []
- 在分支裡若某些路徑漏寫 return，也會回傳 None，可能導致程式邏輯錯誤
- None 是 singleton，整個程式中只有一個實例
- 在條件判斷時，if variable is None: 是檢查「是否真的沒有值」的慣用寫法
- None 在布林運算中視為 False


In [38]:
def append_list(item, lst=None):
    if lst is None:
        lst = []
    lst.append(item)
    return lst

print(append_list(1))

print(append_list(4, [1, 2, 3]))

[1]
[1, 2, 3, 4]


Best practice of None:

In [39]:
# 1. 用途區分：缺值 vs 錯誤
# 當函式「合法地沒有結果」時，回傳 None
# 當發生「不可接受的狀態」或「運算失敗」時，應該丟出例外而不是回傳 None
def find_user(id):
    user = db.query(id)
    if not user:
        return None  # 合法：沒找到就是沒有資料
    return user

def load_config(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} does not exist.")  # 錯誤：直接以例外通知呼叫端
    # ... 讀檔並回傳設定

In [41]:
# 2. 型別提示：明示 Optional
# 搭配 PEP 484 的型別提示，讓團隊或靜態檢查工具一眼就看出哪裡可能是 None
from typing import Optional
import datetime

def parse_date(s: str) -> Optional[datetime.date]:
    try:
        return datetime.strptime(s, "%Y-%m-%d").date()
    except ValueError:
        return None

In [45]:
# 3. 單一實例：None 是 singleton
# - 比較時用 is None 而不是 == None
# - None 在布林上下文會當作 False，但為了可讀性，還是用 is／is not
a = None
b = None
c = 5
print(id(a))
print(id(b))
print(a is b)

if a is None:
    print(f"a is None")

if c is not None:
    print(f"c is not None")

9695488
9695488
True
a is None
c is not None


##4.8 More on Defining Functions

###4.8.1. Default Argument Values

In [46]:
def ask_ok(prompt, retries=4, reminder='Please try again!'):
    while True:
        ok = input(prompt)
        if ok in ('y', 'ye', 'yes'):
            return True
        if ok in ('n', 'no', 'nop', 'nope'):
            return False
        retries = retries - 1
        if retries < 0:
            raise ValueError('invalid user response')
        print(reminder)

In [47]:
# 重要警告：預設值只求值一次。
# 當預設值為可變物件，例如 list、dictionary（字典）或許多類別實例時，會產生不同的結果。
# 例如，以下函式於後續呼叫時會累積曾經傳遞的引數：
def f(a, L=[]):
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

[1]
[1, 2]
[1, 2, 3]


In [49]:
# 如果不想在後續呼叫之間共用預設值，應以如下方式編寫函式：
def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

[1]
[2]
[3]


###4.8.2 Keyword Arguments

In [52]:
def parrot(voltage, state='a stiff', action='voom', type='Norwegian Blue'):
    print("-- This parrot wouldn't", action, end=' ')
    print("if you put", voltage, "volts through it.")
    print("-- Lovely plumage, the", type)
    print("-- It's", state, "!")
    print("\n")

In [53]:
# Valid Call
parrot(1000)                                          # 1 positional argument
parrot(voltage=1000)                                  # 1 keyword argument
parrot(voltage=1000000, action='VOOOOOM')             # 2 keyword arguments
parrot(action='VOOOOOM', voltage=1000000)             # 2 keyword arguments
parrot('a million', 'bereft of life', 'jump')         # 3 positional arguments
parrot('a thousand', state='pushing up the daisies')  # 1 positional, 1 keyword

-- This parrot wouldn't voom if you put 1000 volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


-- This parrot wouldn't voom if you put 1000 volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


-- This parrot wouldn't VOOOOOM if you put 1000000 volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


-- This parrot wouldn't VOOOOOM if you put 1000000 volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


-- This parrot wouldn't jump if you put a million volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's bereft of life !


-- This parrot wouldn't voom if you put a thousand volts through it.
-- Lovely plumage, the Norwegian Blue
-- It's pushing up the daisies !




In [56]:
# Invalid Call
parrot()                     # required argument missing
parrot(voltage=5.0, 'dead')  # non-keyword argument after a keyword argument
parrot(110, voltage=220)     # duplicate value for the same argument
parrot(actor='John Cleese')  # unknown keyword argument

SyntaxError: positional argument follows keyword argument (ipython-input-3113429359.py, line 3)

In [62]:
# 1. kind 是一個必填的位置參數
# 2. *arguments 收集所有多餘的位置參數成為一個 tuple
# 3. **keywords 收集所有多餘的關鍵字參數成為一個 dict
# 4. 呼叫時可用 *、** 解包已有的 list/tuple、dict
# 5. 常見於不定參數函式、decorator、CLI 工具封裝

def cheeseshop(kind, *arguments, **keywords):
    print("-- Do you have any", kind, "?")
    print("-- I'm sorry, we're all out of", kind)
    for arg in arguments:
        print(arg)
    print("-" * 40)
    for kw in keywords:
        print(kw, ":", keywords[kw])

In [63]:
cheeseshop("Limburger",
           "It's very runny, sir.",
           "It's really very, VERY runny, sir.",
           shopkeeper="Michael Palin",
           client="John Cleese",
           sketch="Cheese Shop Sketch")
# 注意，關鍵字引數的輸出順序與呼叫函式時被提供的順序必定一致。

-- Do you have any Limburger ?
-- I'm sorry, we're all out of Limburger
It's very runny, sir.
It's really very, VERY runny, sir.
----------------------------------------
shopkeeper : Michael Palin
client : John Cleese
sketch : Cheese Shop Sketch


In [65]:
# Another example:
def func(kind, *arguments, **keywords):
    print("kind       =", kind)
    print("arguments  =", arguments)
    print("keywords   =", keywords)

func("sum", 1, 2, 3, debug=True, verbose=False)

kind       = sum
arguments  = (1, 2, 3)
keywords   = {'debug': True, 'verbose': False}


In [67]:
args = [10, 20, 30]
kwargs = {"precision": 2, "rounding": "floor"}

func("average", *args, **kwargs)
# 在呼叫時，*args 會把 list 內的元素拆成位置參數
# **kwargs 會把 dict 內的鍵值對拆成關鍵字參數

kind       = average
arguments  = (10, 20, 30)
keywords   = {'precision': 2, 'rounding': 'floor'}


In [71]:
# Usage of typing
from typing import Any

def func(kind: str, *args: Any, **kwargs: Any) -> None:
    print("kind       =", kind)
    print("arguments  =", args)
    print("keywords   =", kwargs)

func("sum", 1, 2, 3, debug=True, verbose=False)

kind       = sum
arguments  = (1, 2, 3)
keywords   = {'debug': True, 'verbose': False}


###4.8.6 Lambda Expressions
lambda 關鍵字用於建立小巧的匿名函式。lambda a, b: a+b 函式返回兩個引數的和。Lambda 函式可用於任何需要函數物件的地方。在語法上，它們被限定只能是單一運算式。在語義上，它就是一個普通函式定義的語法糖 (syntactic sugar)。與巢狀函式定義一樣，lambda 函式可以從包含它的作用域中引用變數：

In [75]:
def make_incrementor(n):
    return lambda x: x + n

# which is equivlant to
def _make_incrementor(n):
    def increment(x):
        return x + n
    return increment

f = make_incrementor(42)    # n = 42
print(f(0))                 # x = 0
print(f(10))                # x = 10
print(f(20))                # x = 20

42
52
62


In [78]:
pairs = [(1, 'one'), (2, 'two'), (3, 'three'), (4, 'four')]
pairs.sort(key=lambda pair: pair[1])
print(pairs)

# which is euqivlant to
def get_second(pair: tuple[int, str]) -> str:
    return pair[1]
pairs = [(1, 'one'), (2, 'two'), (3, 'three'), (4, 'four')]
pairs.sort(key=get_second)
print(pairs)



[(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]
[(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]


###4.8.7
